### Imports

In [1]:
import warnings 
warnings.filterwarnings('ignore')

import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from transformers import SeamlessM4Tv2Model, AutoProcessor

from src.tokenize import AggregatedTokenizer
from src.generate import AggregatedGenerator
from src.evaluate import SimilarityChecker


### Load models 

In [2]:
device = torch.device("cuda:0") 

tokenizer = AutoTokenizer.from_pretrained("facebook/nllb-200-3.3B")
model = AutoModelForSeq2SeqLM.from_pretrained("facebook/nllb-200-3.3B").to(device)

processor = AutoProcessor.from_pretrained("facebook/seamless-m4t-v2-large")
model2 = SeamlessM4Tv2Model.from_pretrained("facebook/seamless-m4t-v2-large").to(device)

tokenizer3 = AutoTokenizer.from_pretrained("google/madlad400-3b-mt")
model3 = AutoModelForSeq2SeqLM.from_pretrained("google/madlad400-3b-mt").to(device)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [3]:
def seamless_tokenizer_postprocessing(decoder_inputs):
    input_ids_data = decoder_inputs['input_ids']
    decoder_inputs['input_ids'] = input_ids_data[(input_ids_data != 0) & (input_ids_data != 3)].unsqueeze(0) 


agg_tokenizer = AggregatedTokenizer(
    tokenizers=[
        tokenizer, 
        processor.tokenizer,
        tokenizer3,
    ],
    tokenization_kwargs=[
        dict(),
        dict(src_lang="eng", tgt_lang="rus"),
        dict(),
    ],
    decoder_tokenization_postprocessing=[
        None,
        seamless_tokenizer_postprocessing,
        None
    ]
)

Adding tokens from tokenizer 0 to aggregated tokenizer --> Done
Adding tokens from tokenizer 1 to aggregated tokenizer --> Done
Adding tokens from tokenizer 2 to aggregated tokenizer --> Done


In [4]:
agg_generator = AggregatedGenerator(
    models=[
        model, 
        model2,
        model3,
    ],
    generation_kwargs=[
        dict(),
        dict(generate_speech=False),
        dict(),

    ],
    agg_tokenizer=agg_tokenizer,
    decoder_prompts = [
        "rus_Cyrl",
        "__rus__",
        None,
    ],
    encoder_prompts = [
        None,
        None,
        "<2ru>"
    ]
)

In [5]:
similarity_checker = SimilarityChecker(score_names=['bertscore'])

### Set input text [eng] 

In [6]:
encoder_input_text = """
Virgin Australia, the trading name of Virgin Australia Airlines Pty Ltd, 
is an Australian-based airline. It is the largest airline by fleet size 
to use the Virgin brand. It commenced services on 31 August 2000 as 
Virgin Blue, with two aircraft on a single route. It suddenly found 
itself as a major airline in Australia's domestic market after the 
collapse of Ansett Australia in September 2001. The airline has since 
grown to directly serve 32 cities in Australia, from hubs in Brisbane, 
Melbourne and Sydney."
"""

### Get individual translations of models [rus]

In [7]:
all_single_translations = agg_generator.generate_all_single(encoder_input_text, device=device)
all_single_translations

['Virgin Australia, торговое название Virgin Australia Airlines Pty Ltd, является австралийской авиакомпанией. Это крупнейшая авиакомпания по размеру флота, использующая бренд Virgin. Она начала обслуживание 31 августа 2000 года как Virgin Blue, с двумя самолетами на одном маршруте. Она внезапно оказалась крупной авиакомпанией на внутреннем рынке Австралии после краха Ansett Australia в сентябре 2001 года.',
 'Virgin Australia, торговое название Virgin Australia Airlines Pty Ltd, является австралийской авиакомпанией, крупнейшей авиакомпанией по размеру флота, использующей бренд Virgin. Она начала обслуживать 31 августа 2000 года как Virgin Blue, с двумя самолетами на одном маршруте. Она внезапно стала крупной авиакомпанией на внутреннем рынке Австралии после краха Ansett Australia в сентябре 2001 года.',
 'Virgin Australia, торговое название Virgin Australia Airlines Pty Ltd, является австралийской авиакомпанией. Это самая большая авиакомпания по размеру флота, использующая бренд Virgi

### Get ensemble of models translation [rus]

In [11]:
ensemble_tranlation = agg_generator.generate_agg(encoder_input_text, num_beams=3, max_new_tokens=256, device=device)
ensemble_tranlation


score = -0.30709 - Virgin
score = -3.61865 - "
score = -4.71748 - Компа


score = -0.48457 - Virgin Australia
score = -5.01384 - "Vir
score = -5.43200 - Компания


score = -1.26246 - Virgin Australia,
score = -2.46060 - Virgin Australia (
score = -3.17478 - Virgin Australia -


score = -2.10347 - Virgin Australia, тор
score = -2.73364 - Virgin Australia, торгов
score = -3.94866 - Virgin Australia, торго


score = -2.69358 - Virgin Australia, торгово
score = -3.35490 - Virgin Australia, торг
score = -4.05134 - Virgin Australia, торговое


score = -3.17872 - Virgin Australia, торговое
score = -4.45430 - Virgin Australia, торгово-
score = -4.51827 - Virgin Australia, торг.


score = -4.05530 - Virgin Australia, торговое назва
score = -4.43530 - Virgin Australia, торговое название
score = -5.68575 - Virgin Australia, торг. назва


score = -4.19998 - Virgin Australia, торговое название
score = -4.64851 - Virgin Australia, торговое название Virgin
score = -5.88285 - Virgin Australia, торг. 

Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x7f8ed77d9cc0>>
Traceback (most recent call last):
  File "/home/sharkov.sergey2/.local/lib/python3.10/site-packages/ipykernel/ipkernel.py", line 770, in _clean_thread_parent_frames
    def _clean_thread_parent_frames(
KeyboardInterrupt: 



score = -125.54559 - Virgin Australia, торговое название Virgin Australia Airlines Pty Ltd, является австралийской авиакомпанией. Это крупнейшая авиакомпания по размеру флота, использующая бренд Virgin. Она начала свои услуги 31 августа 2000 года как Virgin Blue, с двумя самолетами на одном маршруте. Она внезапно оказалась крупной авиакомпанией на внутреннем рынке Австралии после краха Ansett Australia в сентябре 2001 года. С тех пор авиакомпания выросла, чтобы напряму обслужвать 32 города в Австралии, из узлов в Брисбене, Мельбурне и Сиднее. "ВИДЕО: Virgin Australia Airlines Pty Ltd, Virgin Australia, Virgin Australia, Virgin Australia, Virgin Australia, Virgin Australia, Virgin Australia, Virgin Australia, Virgin Australia, Virgin Australia, Virgin Australia, Virgin Australia, Virgin Australia, Virgin Australia,
score = -125.85474 - Virgin Australia, торговое название Virgin Australia Airlines Pty Ltd, является австралийской авиакомпанией. Это крупнейшая авиакомпания по размеру флот

Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x7f8ed77d9cc0>>
Traceback (most recent call last):
  File "/home/sharkov.sergey2/.local/lib/python3.10/site-packages/ipykernel/ipkernel.py", line 770, in _clean_thread_parent_frames
    def _clean_thread_parent_frames(
KeyboardInterrupt: 


KeyboardInterrupt: 

### Calculate group similarity score

In [10]:
similarity_checker.check_similarity(texts=all_single_translations)

{'bertscore': {'precision': 0.9621651768684387,
  'recall': 0.9119097391764323,
  'f1': 0.9359677235285441}}